In [2]:
from pyspark.sql.functions import col
from pyspark.sql.functions import sum
from pyspark.sql.functions import count
from pyspark.sql import SparkSession
from delta.tables import *
import datetime
import pytz

spark = (
    SparkSession
    .builder
    .appName('Lakehouse_OlhoVivo_Raw_To_Trusted')
    .config('spark.sql.extensions','oi.delta.DeltaSparkSessionExtension')
    .config('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog')
    .getOrCreate()
)
spark

In [17]:
path_trusted = 's3a://trusted/olho-vivo/linhas/buscar-linhas/'
print(f"Path TRUSTED: {path_trusted}")

Path TRUSTED: s3a://trusted/olho-vivo/linhas/buscar-linhas/


In [18]:
df_linhas = spark.read.parquet(path_trusted)
df_linhas.show(5)
df_linhas.printSchema()

+-----+-----+
|   cl|   lc|
+-----+-----+
|  412|false|
|  413|false|
|  415|false|
| 2562|false|
|33180|false|
+-----+-----+
only showing top 5 rows

root
 |-- cl: long (nullable = true)
 |-- lc: boolean (nullable = true)



In [13]:
total_records = df_linhas.count()
print(total_records)

4


In [14]:
df_linhas.select("cl", "lc").show()

+-----+----+
|   cl|  lc|
+-----+----+
| 1726|true|
|34494|true|
|34494|true|
| 1726|true|
+-----+----+



In [29]:
path_linhas = 's3a://trusted/olho-vivo/linhas/buscar-linhas/'
path_posicao_veiculos = 's3a://trusted/olho-vivo/posicao-veiculos/buscar-posicao/'

print(f"Path LINHAS: {path_linhas}")
print(f"Path PARADAS: {path_posicao_veiculos}")

Path LINHAS: s3a://trusted/olho-vivo/linhas/buscar-linhas/
Path PARADAS: s3a://trusted/olho-vivo/posicao-veiculos/buscar-posicao/


In [50]:
from delta.tables import DeltaTable

# Load Delta Tables
df_linhas = spark.read.format("delta").load(path_linhas)
df_posicao = spark.read.format("delta").load(path_posicao_veiculos)
df_linhas.show(5)
df_posicao.show(5)


+-----+-----+
|   cl|   lc|
+-----+-----+
|  412|false|
|  413|false|
|  415|false|
| 2562|false|
|33180|false|
+-----+-----+
only showing top 5 rows

+-----+---+-------+
| vs_p| cl|      c|
+-----+---+-------+
|71255|  1|5119-10|
|75503|  4|669A-10|
|72917|  5|6801-10|
|85617|  6|6837-10|
|72771| 12|6835-10|
+-----+---+-------+
only showing top 5 rows



In [52]:
df_joined = df_linhas.join(df_posicao, df_linhas["cl"] == df_posicao["cl"], "inner")
df_joined.show()



+-----+-----+-----+-----+-------+
|   cl|   lc| vs_p|   cl|      c|
+-----+-----+-----+-----+-------+
| 2562|false|85499| 2562|8030-10|
|35330|false|85550|35330|8030-10|
+-----+-----+-----+-----+-------+



In [53]:
df_joined = df_linhas.join(
    df_posicao, df_linhas["cl"] == df_posicao["cl"], "inner"
).select(
    df_linhas["cl"], 
    df_linhas["lc"], 
    df_posicao["vs_p"]
)

df_joined.show()

+-----+-----+-----+
|   cl|   lc| vs_p|
+-----+-----+-----+
| 2562|false|85499|
|35330|false|85550|
+-----+-----+-----+

